In [71]:
import pandas as pd
import numpy as np
import pickle
from sklearn.preprocessing import OneHotEncoder, StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split

In [72]:
data = pd.read_csv('Churn_Modelling.csv')
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


## Data Preprocessing

In [73]:
data.drop(['RowNumber',	'CustomerId', 'Surname'], axis = 1, inplace=True)
data.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [74]:
labelEncoder = LabelEncoder()
data['Gender'] = labelEncoder.fit_transform(data['Gender'])
data.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0


In [75]:
oneHotEncoder = OneHotEncoder()
geo_encoder = oneHotEncoder.fit_transform(data[['Geography']])
geo_encoder = geo_encoder.toarray()
geo_encoder

array([[1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       ...,
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.]])

In [76]:
oneHotEncoder.get_feature_names_out(['Geography'])

array(['Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [77]:
geo_encoder_df = pd.DataFrame(geo_encoder, columns=oneHotEncoder.get_feature_names_out(['Geography']))
geo_encoder_df.head()

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0


In [78]:
data.drop(['Geography'], axis = 1, inplace=True)
data = pd.concat([data, geo_encoder_df], axis=1)
data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [79]:
data.shape

(10000, 13)

In [80]:
with open('labelEncoder_gender.pkl', 'wb') as file:
    pickle.dump(labelEncoder, file)

with open('oneHotEncoder_geo.pkl', 'wb') as file:
    pickle.dump(oneHotEncoder, file)

## Divide independent and. dependent features


In [81]:
X = data.drop(['Exited'], axis=1)
Y = data['Exited']

X.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0.0,0.0,1.0


## Split the data into training and testing 

In [82]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)
print(X_train.shape, X_test.shape, Y_train.shape, Y_test.shape)

(8000, 12) (2000, 12) (8000,) (2000,)


In [83]:
X_train.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
9254,686,1,32,6,0.00,2,1,1,179093.26,1.0,0.0,0.0
1561,632,1,42,4,119624.60,2,1,1,195978.86,0.0,1.0,0.0
1670,559,1,24,3,114739.92,1,1,0,85891.02,0.0,0.0,1.0
6087,561,0,27,9,135637.00,1,1,0,153080.40,1.0,0.0,0.0
6669,517,1,56,9,142147.32,1,0,0,39488.04,1.0,0.0,0.0


In [84]:
X_test.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
6252,596,1,32,3,96709.07,2,0,0,41788.37,0.0,1.0,0.0
4684,623,1,43,1,0.00,2,1,1,146379.30,1.0,0.0,0.0
1731,601,0,44,4,0.00,2,1,0,58561.31,0.0,0.0,1.0
4742,506,1,59,8,119152.10,2,1,1,170679.74,0.0,1.0,0.0
4521,560,0,27,7,124995.98,1,1,1,114669.79,0.0,0.0,1.0


## StandardScaler - Scales data to have a mean of 0 and a standard deviation of 1

In [85]:
## Scales data to have a mean of 0 and a standard deviation of 1
scalar = StandardScaler() 
X_train = scalar.fit_transform(X_train)
X_test = scalar.fit_transform(X_test)

In [86]:
print(len(X_train[0]))
print(len(X_train))

12
8000


In [87]:
with open('scalar.pkl', 'wb') as file:
    pickle.dump(scalar, file)

## Applying ANN to the training and testing 

In [88]:
import tensorflow
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
import datetime

In [89]:
model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1], )), ## This is input and HL1
    Dense(32, activation='relu'), ## HL2
    Dense(1, activation='sigmoid') ## Output layer
])

/Users/denesh/Desktop/Learning/Generative-AI-With-Langchain-and-Huggingface/venv/lib/python3.12/site-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [90]:
model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_6 (Dense)                 │ (None, 64)             │           832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,945 (11.50 KB)

 Trainable params: 2,945 (11.50 KB)

 Non-trainable params: 0 (0.00 B)

In [91]:
opt = tensorflow.keras.optimizers.Adam(learning_rate=0.001)
loss = tensorflow.keras.losses.BinaryCrossentropy()
loss

<LossFunctionWrapper(<function binary_crossentropy at 0x17fba7ec0>, kwargs={'from_logits': False, 'label_smoothing': 0.0, 'axis': -1})>

In [92]:
model.compile(optimizer=opt, loss = loss, metrics = ['accuracy'])

In [93]:
log_dirs = 'logs/fit/'+ datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callbacks = TensorBoard(log_dir=log_dirs, histogram_freq=1)
tensorboard_callbacks

In [94]:
earlystopping_callbacks = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

In [95]:
history=model.fit(
    X_train,Y_train,validation_data=(X_test,Y_test),epochs=100,
    callbacks=[tensorboard_callbacks,earlystopping_callbacks]
)

Epoch 1/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7996 - loss: 0.4927 - val_accuracy: 0.8340 - val_loss: 0.3991
Epoch 2/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 749us/step - accuracy: 0.8390 - loss: 0.3978 - val_accuracy: 0.8545 - val_loss: 0.3575
Epoch 3/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 747us/step - accuracy: 0.8478 - loss: 0.3608 - val_accuracy: 0.8585 - val_loss: 0.3471
Epoch 4/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 743us/step - accuracy: 0.8554 - loss: 0.3485 - val_accuracy: 0.8560 - val_loss: 0.3515
Epoch 5/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 741us/step - accuracy: 0.8593 - loss: 0.3367 - val_accuracy: 0.8565 - val_loss: 0.3470
Epoch 6/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 748us/step - accuracy: 0.8654 - loss: 0.3258 - val_accuracy: 0.8630 - val_loss: 0.3461
Epoch 7/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 728us/step - accuracy: 0.8616 - loss: 0.3361 - val_accuracy: 0.8545 - val_loss: 0.3503
Epoch 8/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 733us/step - accuracy: 0.8689 - loss: 0.3

In [96]:
import os

fileName = 'ann_model.h5'
if os.path.exists(fileName):
    os.remove(fileName)
    
model.save('ann_model.h5')

In [97]:
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [98]:
%tensorboard --logdir logs/fit

Reusing TensorBoard on port 6006 (pid 44423), started 0:31:18 ago. (Use '!kill 44423' to kill it.)